<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Spark の LightGBM を使用したコンテンツベースのパーソナライゼーション

このノートブックでは、コンテンツベースのパーソナライゼーション シナリオで [MMLSpark](https://github.com/Azure/mmlspark) を使用して Spark で [LightGBM](https://github.com/Microsoft/Lightgbm) モデルをトレーニングする方法の簡単な例を示します。

クリックスルー率(CTR)の最適化に使用できるウェブサイト広告のよく知られたデータセットである[CRITEO データセット](https://www.kaggle.com/c/criteo-display-ad-challenge)を使用しています。データセットは、一連の数値およびカテゴリのフィーチャーと、新たに追加されたクリックされたかどうかを示すバイナリ ラベルで構成されています。

モデルは[LightGBM](https://github.com/Microsoft/Lightgbm)に基づいており、ツリーベースの学習アルゴリズムを使用するグラデーションブーストフレームワークです。最後に、
[MMLSpark](https://github.com/Azure/mmlspark) ライブラリを使用することで、LightGBM を Spark 環境で呼び出し、分散して計算できます。

このシナリオは**暗黙的なフィードバック**の良い例であり、バイナリラベルはユーザーとアイテムの間の相互作用を示します。これは、ユーザーがコンテンツを明示的に評価する明示的なフィードバック (1 から 5 など) とは対照的です。


## グローバル設定とインポート

このノートブックは、DSVM または Azure Databricks の Spark 環境で実行できます。インストールプロセスの詳細については、[セットアップ手順](../../SETUP.md)を参照してください。

**Azure Databricksでの注意:**

* Azure Databricks で正しい依存関係の設定を簡略化するために、Python スクリプトが用意されています。詳細については、```python scripts/databricks_install.py -h``` を実行してください。
* MMLSpark は、自動スケールが有効になっているクラスターで実行しないでください。このノートブックを実行する前に、Azure Databricks クラスター構成でフラグを無効にします。

In [1]:
import os
import sys

sys.path.append("../../")

import pyspark
from pyspark.ml import PipelineModel
from pyspark.ml.feature import FeatureHasher
import papermill as pm

from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.common.notebook_utils import is_databricks
from reco_utils.dataset.criteo import load_spark_df
from reco_utils.dataset.spark_splitters import spark_random_split

# Setup MML Spark
if not is_databricks():
    # databricks_install スクリプトから MML Spark の maven コーディネートを取得する
    from scripts.databricks_install import MMLSPARK_INFO
    packages = [MMLSPARK_INFO["maven"]["coordinates"]]
    repo = MMLSPARK_INFO["maven"].get("repo")
    spark = start_or_get_spark(packages=packages, repository=repo)
    dbutils = None
    print("MMLSpark version: {}".format(MMLSPARK_INFO['maven']['coordinates']))

from mmlspark import ComputeModelStatistics
from mmlspark import LightGBMClassifier

print("System version: {}".format(sys.version))
print("PySpark version: {}".format(pyspark.version.__version__))


MMLSpark version: com.microsoft.ml.spark:mmlspark_2.11:0.16.dev8+2.g6a5318b
System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
PySpark version: 2.3.0


In [2]:
# Criteo データのサイズ。"sample" または "full" で指定
DATA_SIZE = "sample"

# LightGBM パラメータ
# パラメータの詳細: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
NUM_LEAVES = 32
NUM_ITERATIONS = 50
LEARNING_RATE = 0.1
FEATURE_FRACTION = 0.8
EARLY_STOPPING_ROUND = 10

# モデル名
MODEL_NAME = 'lightgbm_criteo.mml'

## データ準備
 [Criteo Display Advertising Challenge](https://www.kaggle.com/c/criteo-display-ad-challenge) (Criteo DAC) データセットは、CTR 予測モデルを開発するための業界のベンチマーク データセットとしてよく知られており、研究論文で頻繁に使用されています。元のデータセットには 45 百万行を超える行が含まれていますが、100,000 行を持つダウンサンプリングされたデータセットもあります (これは DATA_SIZE = "sample"を設定することで使用できます)。各行は Criteo が提供する表示広告に対応し、最初の列は、この広告がクリックされたかどうかを示します。

データセットには 1 つのラベル列と 39 個のフィーチャ列があり、13 列が整数値 (int00-int12) で、26 列がカテゴリ フィーチャ (cat00-cat25) です。

列が表すものは提供されませんが、この場合、整数値とカテゴリ値は、ユーザーおよび/または項目の内容を表す特徴と見なすことができます。ラベルはバイナリであり、ユーザーのアイテムとの対話を示す暗黙的なフィードバックの例です。このデータセットを使用すると、使用可能なユーザーおよびアイテムコンテンツ機能に基づいて、ユーザーがアイテムを操作する確率を予測するモデルを構築する方法を示すことができます。

In [3]:
raw_data = load_spark_df(size=DATA_SIZE, spark=spark, dbutils=dbutils)
# データの可視化
raw_data.limit(2).toPandas().head()

8.79MB [00:00, 32.6MB/s]                            


,label,int00,int01,int02,int03,int04,int05,int06,int07,int08,...,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25
0,0,1,1,5,0,1382,4,15,2,181,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,None,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2,0,44,1,102,8,2,2,4,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,None,3a171ecb,43f13e8b,e8b83407,731c3655


### フィーチャーの処理
提供されるフィーチャー データには、整数とカテゴリ フィーチャー フィールドの両方で多くの欠損値があります。さらに、カテゴリ フィーチャーには多くの異なる値が含まれるため、フィーチャ データを効果的にクリーニングして表現することは、モデルをトレーニングする前に重要な手順です。

値が欠落しているフィーチャーと高いカーディナリティを持つ両方のフィーチャを管理する最も簡単な方法の 1 つは、ハッシュトリックを使用することです。[FeatureHasher](http://spark.apache.org/docs/latest/ml-features.html#featurehasher) トランスフォーマーは整数値を渡し、カテゴリ フィーチャを低次元のスパース ベクトルにハッシュします。

まず、トレーニングとテストのためにデータセットがランダムに分割され、各データセットに機能処理が適用されます。

In [4]:
raw_train, raw_test = spark_random_split(raw_data, ratio=0.8, seed=42)

In [5]:
columns = [c for c in raw_data.columns if c != 'label']
feature_processor = FeatureHasher(inputCols=columns, outputCol='features')

In [6]:
train = feature_processor.transform(raw_train)
test = feature_processor.transform(raw_test)

## モデルトレーニング
MMLSpark では、`LightGBMClassifier` クラスを使用して、バイナリ分類の LightGBM 実装が呼び出され、目的変数を `バイナリ` として指定します。この場合、正のラベルの出現率は非常に低いので、`isUnbalance` フラグを true に設定すると、この不均衡を緩和するのに役立ちます。<br><br>

### ハイパーパラメータ
Spark で LightGBM 分類器をトレーニングするための主要な[ハイパーパラメーター](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters-Tuning.rst)の一部を次に示します:

- `numLeaves`: 各ツリーの葉の数
- `numIterations`: ブーストを適用する反復回数
- `learningRate`: ツリー間でのトレーニングの学習率
- `featureFraction`: ツリーのトレーニングに使用されるフィーチャーの割合
- `earlyStoppingRound`: オーバーフィットを避けるために早期停止を適用できるラウンド

In [7]:
lgbm = LightGBMClassifier(
    labelCol="label",
    featuresCol="features",
    objective="binary",
    isUnbalance=True,
    boostingType="gbdt",
    boostFromAverage=True,
    baggingSeed=42,
    numLeaves=NUM_LEAVES,
    numIterations=NUM_ITERATIONS,
    learningRate=LEARNING_RATE,
    featureFraction=FEATURE_FRACTION,
    earlyStoppingRound=EARLY_STOPPING_ROUND
)

### モデルのトレーニングと評価

In [8]:
model = lgbm.fit(train)
predictions = model.transform(test)

In [9]:
evaluator = (
    ComputeModelStatistics()
    .setScoredLabelsCol("prediction")
    .setLabelCol("label")
    .setEvaluationMetric("AUC")
)

result = evaluator.transform(predictions)
auc = result.select("AUC").collect()[0][0]
result.show()

+---------------+------------------+
|evaluation_type|               AUC|
+---------------+------------------+
| Classification|0.6870253907336659|
+---------------+------------------+



In [10]:
# テストで使用した paermill の結果の記録
pm.record("auc", auc)

## モデルの保存
フィーチャー処理やモデル予測を含む生データを操作するための完全なパイプラインは、別のワークフローで使用するために保存および再ロードできます。

In [ ]:
# モデルの保存
pipeline = PipelineModel(stages=[feature_processor, model])
pipeline.write().overwrite().save(MODEL_NAME)

## 参考文献
\[1\] Guolin Ke, Qi Meng, Thomas Finley, Taifeng Wang, Wei Chen, Weidong Ma, Qiwei Ye, and Tie-Yan Liu. 2017. LightGBM: A highly efficient gradient boosting decision tree. In Advances in Neural Information Processing Systems. 3146–3154. https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf <br>
\[2\] MML Spark: https://mmlspark.blob.core.windows.net/website/index.html <br>
